# Model Training


In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow import keras
from tensorflow.keras import layers


In [38]:
player_stats = pd.read_csv('../tmp/02/player_stats_points.csv')

player_stats = player_stats.drop(['name'], axis=1)


x_data = player_stats.iloc[:, :-1]
y_data = player_stats[['points']]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, shuffle=True, random_state=42) 


In [44]:
print("TensorFlow version:", tf.__version__)

scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)
print(X_train.shape)

model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(78, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)  # Output layer
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

history = model.fit(X_train, y_train, epochs=1000, batch_size=8, validation_split=0.1, verbose=1)

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")







TensorFlow version: 2.18.0
(130, 78)
Epoch 1/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 270.9405 - mae: 13.1413 - val_loss: 126.6112 - val_mae: 9.0916
Epoch 2/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 182.2522 - mae: 10.5437 - val_loss: 104.5792 - val_mae: 8.3977
Epoch 3/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 186.1572 - mae: 10.6497 - val_loss: 68.7694 - val_mae: 6.9733
Epoch 4/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 167.5639 - mae: 10.1078 - val_loss: 32.9124 - val_mae: 4.8079
Epoch 5/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 101.1408 - mae: 8.0638 - val_loss: 24.4868 - val_mae: 4.3143
Epoch 6/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 68.4435 - mae: 6.8744 - val_loss: 29.9364 - val_mae: 4.5388
Epoch 7/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50.8287 - mae: 5.6768 - val_loss: 33.3361 - val_mae: 4.7991
Epoch 8/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 49.3292 - mae: 5.8556 - val_loss: 25.7974 - val_mae:

## Use Test Data

In [40]:
# Make predictions
predictions = model.predict(X_test)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error (MAE): {mae}")

# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error (MSE): {mse}")

# Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# R-squared (R²)
r2 = r2_score(y_test, predictions)
print(f"R-squared (R²): {r2}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Mean Absolute Error (MAE): 4.534502592722574
Mean Squared Error (MSE): 34.7597997942277
Root Mean Squared Error (RMSE): 5.895744210379865
R-squared (R²): 0.61977219581604


/Users/achester/Repos/DSC/412/Project/DSC412-project-amcheste/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [43]:
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(78, activation='relu'),
    layers.Dense(32, activation='swish'),
    layers.Dense(16, activation='swish'),
    layers.Dense(1)  # Output layer
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

history = model.fit(X_train, y_train, epochs=1000, batch_size=8, validation_split=0.1, verbose=1)

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

Epoch 1/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 219.9351 - mae: 11.6353 - val_loss: 107.7785 - val_mae: 8.4445
Epoch 2/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 198.9623 - mae: 10.7743 - val_loss: 70.5938 - val_mae: 6.9762
Epoch 3/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 137.4819 - mae: 8.8403 - val_loss: 31.2224 - val_mae: 4.7137
Epoch 4/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 104.5257 - mae: 7.7328 - val_loss: 27.4645 - val_mae: 4.4174
Epoch 5/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 78.8625 - mae: 7.1919 - val_loss: 35.3194 - val_mae: 4.9103
Epoch 6/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 72.1633 - mae: 6.9775 - val_loss: 29.7275 - val_mae: 4.5489
Epoch 7/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51.1264 - mae: 5.7619 - val_loss: 32.8986 - val_mae: 4.7072
Epoch 8/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 58.5610 - mae: 6.2033 - val_loss: 31.8473 - val_mae: 4.6105
Epoch 9/1000
15/15 ━━━━━━━━━━━━━━

In [46]:
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(78, activation='relu'),
    layers.Dense(32, activation='swish'),
    layers.Dense(16, activation='swish'),
    layers.Dense(8, activation='swish'),
    layers.Dense(1)  # Output layer
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

history = model.fit(X_train, y_train, epochs=1000, batch_size=8, validation_split=0.1, verbose=1)

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

Epoch 1/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 241.6012 - mae: 12.0762 - val_loss: 105.1267 - val_mae: 8.4618
Epoch 2/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 198.0940 - mae: 10.8566 - val_loss: 50.5481 - val_mae: 5.9504
Epoch 3/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 128.7238 - mae: 8.1758 - val_loss: 37.4012 - val_mae: 5.1512
Epoch 4/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 78.3101 - mae: 7.3699 - val_loss: 35.5051 - val_mae: 5.0154
Epoch 5/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 66.1602 - mae: 6.7603 - val_loss: 28.3148 - val_mae: 4.5364
Epoch 6/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 55.8113 - mae: 6.0560 - val_loss: 34.4542 - val_mae: 4.6990
Epoch 7/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 54.0656 - mae: 6.0633 - val_loss: 30.2590 - val_mae: 4.2637
Epoch 8/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 58.3106 - mae: 6.3573 - val_loss: 26.5185 - val_mae: 4.0124
Epoch 9/1000
15/15 ━━━━━━━━━━━━━━━